# Гібридна постквантова криптографічна схема цифрового підпису ECDSA + ML-DSA

## 1. Імпорт

In [1]:
import sys
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, str(Path.cwd()))

# Імпортуємо нашу гібридну схему
from hybrid import create_hybrid

print("    Імпорт успішний!")

liboqs-python faulthandler is disabled
    Імпорт успішний!


## 2. Ініціалізація гібридної схеми

In [2]:
# Створюємо гібридну схему
hybrid = create_hybrid()

print("    Гібридна схема ініціалізована")

    ML-DSA ініціалізовано (використовується ML-DSA-44)
    Гібридна схема ініціалізована


## 3. Генерація гібридної пари ключів

In [4]:
# Генеруємо пару ключів
keypair = hybrid.generate_keypair()

print("\n     Інформація про ключі:")
print(f"\n     ECDSA P-256:")
ecdsa_priv = hybrid.ecdsa.private_key_to_bytes(keypair.ecdsa_private_key)
ecdsa_pub = hybrid.ecdsa.public_key_to_bytes(keypair.ecdsa_public_key)
print(f"        Приватний: {len(ecdsa_priv)} байт")
print(f"        Публічний: {len(ecdsa_pub)} байт")
print(f"\n     ML-DSA-44:")
print(f"        Приватний: {len(keypair.mldsa_private_key)} байт")
print(f"        Публічний: {len(keypair.mldsa_public_key)} байт")

     Час генерації ключів ECDSA: 0.56 мсек
     Час генерації ключів ML-DSA-44: 0.38 мсек

     Інформація про ключі:

     ECDSA P-256:
        Приватний: 32 байт
        Публічний: 64 байт

     ML-DSA-44:
        Приватний: 2560 байт
        Публічний: 1312 байт


## 4. Створення гібридного підпису

In [5]:
# Дані для підпису
message = b"Simplicity is a difficult thing to achieve."

print(f"Дані: {message.decode('utf-8')}")
print(f"Розмір: {len(message)} байт")

# Створюємо підпис
signature = hybrid.sign(message, keypair)

print(f"\n Підпис створено")
print(f"  ECDSA: {len(signature.ecdsa_signature)} байт")
print(f"  ML-DSA: {len(signature.mldsa_signature)} байт")

Дані: Simplicity is a difficult thing to achieve.
Розмір: 43 байт
     Час підпису за допомогою ECDSA: 0.32 мсек
     Час підпису за допомогою ML-DSA-44: 0.29 мсек

 Підпис створено
  ECDSA: 64 байт
  ML-DSA: 2420 байт


## 5. Верифікація підпису

In [7]:
# Верифікуємо підпис
valid = hybrid.verify(message, signature, keypair)

# Виведення результатів
print("\nЧас перевірки кожного підпису:")
ecdsa_valid = hybrid.ecdsa.verify(message, signature.ecdsa_signature, keypair.ecdsa_public_key)
mldsa_valid = hybrid.mldsa.verify(message, signature.mldsa_signature, keypair.mldsa_public_key)

print("\n" + "="*70)
print("РЕЗУЛЬТАТИ ВЕРИФІКАЦІЇ")
print("="*70)
print(f"ECDSA P-256:   {' ВАЛІДНИЙ' if ecdsa_valid else ' НЕВАЛІДНИЙ'}")
print(f"ML-DSA-44:     {' ВАЛІДНИЙ' if mldsa_valid else ' НЕВАЛІДНИЙ'}")
print(f"Гібридний:     {' ВАЛІДНИЙ' if valid else ' НЕВАЛІДНИЙ'}")
print("="*70)

    Час верифікації гібридного підпису:
     Час перевірки підпису ECDSA: 0.85 мсек
     Час перевірки підпису ML-DSA-44: 0.09 мсек
     Час перевірки гібридного підпису: 1.50 мсек

Час перевірки кожного підпису:
     Час перевірки підпису ECDSA: 2.46 мсек
     Час перевірки підпису ML-DSA-44: 0.12 мсек

РЕЗУЛЬТАТИ ВЕРИФІКАЦІЇ
ECDSA P-256:    ВАЛІДНИЙ
ML-DSA-44:      ВАЛІДНИЙ
Гібридний:      ВАЛІДНИЙ


## 6.: Експорт у JSON

In [6]:
# Експорт підпису
json_signature = hybrid.export_signature(signature)

print("Підпис у JSON форматі:")
print(json_signature[:500] + "...")

# Збереження
with open('signature_demo.json', 'w') as f:
    f.write(json_signature)

print("\n✓ Збережено: signature_demo.json")

Підпис у JSON форматі:
{
  "algorithm": "ECDSA-P256 + ML-DSA-44 + SHA-256",
  "timestamp": "2026-02-21T02:08:06.267128",
  "data_hash_sha256": "40635f091ee200571f2a379d1b924aa5e47c8ba261ace37350e841de80c98437",
  "ecdsa_p256": {
    "signature": "c3a5101e632e0436f44d264a44e920e87f03c520dc49af6b17d51972f902dde31d499b3ec8027cd3410ae9793a7ede0e18ca18477fe965c3295a3957495a2c90"
  },
  "mldsa44": {
    "signature": "53de953c9a962205a2bf9eeea92c14368cfa64a233e0fd45514c9beae6d4d83806fe006a02ec5ccdd488f616779d402e18066fae0e9c...

✓ Збережено: signature_demo.json


## 7. Тест виявлення підміни

In [7]:
print("="*70)
print("ТЕСТ ВИЯВЛЕННЯ ПІДМІНИ ДАНИХ")
print("="*70)

# Оригінальні дані
original = b"Payment amount: 1000 USD"
print(f"\n1. Оригінал: {original.decode()}")

# Створюємо підпис
sig = hybrid.sign(original, keypair)

# Верифікація оригіналу
res1 = hybrid.verify(original, sig, keypair)
print(f"   Результат: {' ВАЛІДНИЙ' if res1 else ' НЕВАЛІДНИЙ'}")

# Підміна даних
tampered = b"Payment amount: 10000 USD"
print(f"\n2. Підміна: {tampered.decode()}")
print("   (змінено 1000 → 10000)")

# Верифікація підроблених даних
res2 = hybrid.verify(tampered, sig, keypair)
print(f"   Результат: {' НЕВАЛІДНИЙ (очікувано)' if not res2 else ' ВАЛІДНИЙ (!!!)'}")

if not res2:
    print("\n ПІДМІНУ ВИЯВЛЕНО!")
    print("  Гібридна схема забезпечує цілісність даних.")

print("\n" + "="*70)

ТЕСТ ВИЯВЛЕННЯ ПІДМІНИ ДАНИХ

1. Оригінал: Payment amount: 1000 USD
     Час підпису за допомогою ECDSA: 0.28 мсек
     Час підпису за допомогою ML-DSA-44: 0.16 мсек
    Час верифікації гібридного підпису:
     Час перевірки підпису ECDSA: 1.21 мсек
     Час перевірки підпису ML-DSA-44: 0.04 мсек
     Час перевірки гібридного підпису: 1.33 мсек
   Результат:  ВАЛІДНИЙ

2. Підміна: Payment amount: 10000 USD
   (змінено 1000 → 10000)
    Час верифікації гібридного підпису:
     Час перевірки підпису ML-DSA-44: 0.07 мсек
     Час перевірки гібридного підпису: 1.69 мсек
   Результат:  НЕВАЛІДНИЙ (очікувано)

 ПІДМІНУ ВИЯВЛЕНО!
  Гібридна схема забезпечує цілісність даних.



## 8. Порівняння розмірів

In [8]:
print("="*70)
print("ПОРІВНЯННЯ РОЗМІРІВ")
print("="*70)

# Розміри підписів
ecdsa_size = len(signature.ecdsa_signature)
mldsa_size = len(signature.mldsa_signature)

print(f"\nECDSA P-256:   {ecdsa_size:>6} байт")
print(f"ML-DSA-44:     {mldsa_size:>6} байт")
print(f"{'-'*40}")
print(f"Гібридний:     {ecdsa_size + mldsa_size:>6} байт")

print(f"\nJSON експорт:  {len(json_signature):>6} байт")

# Розміри ключів
print(f"\n\nПублічні ключі:")
print(f"ECDSA P-256:   {len(ecdsa_pub):>6} байт")
print(f"ML-DSA-44:     {len(keypair.mldsa_public_key):>6} байт")
print(f"{'-'*40}")
print(f"Гібридний:     {len(ecdsa_pub) + len(keypair.mldsa_public_key):>6} байт")

print("\n" + "="*70)

ПОРІВНЯННЯ РОЗМІРІВ

ECDSA P-256:       64 байт
ML-DSA-44:       2420 байт
----------------------------------------
Гібридний:       2484 байт

JSON експорт:    5239 байт


Публічні ключі:
ECDSA P-256:       64 байт
ML-DSA-44:       1312 байт
----------------------------------------
Гібридний:       1376 байт



## 9. Характеристики безпеки

In [9]:
print("="*70)
print("ХАРАКТЕРИСТИКИ БЕЗПЕКИ")
print("="*70)

print("\nECDSA P-256 (NIST FIPS 186-5):")
print("   - Крива: NIST P-256 (secp256r1)")
print("   - Поле: GF(p) де p ≈ 2^256")
print("   - Хеш: SHA-256")
print("   - Класична безпека: ~128 біт")
print("   - Квантова вразливість: Повністю зламується (Шор)")
print("   - Стандарт: Міжнародний NIST")

print("\nML-DSA-44 (NIST FIPS 204):")
print("   - Базис: Module Lattice")
print("   - Рівень: NIST Level 2")
print("   - Безпека: ~128 біт класичної")
print("   - Стійкість: Квантові атаки (128 біт)")
print("   - Стандарт: FIPS 204 (2024)")

print("\nГібридна схема:")
print("  - Верифікація: ОБИДВА підписи мають бути валідні")
print("  - Захист: Класичний + Постквантовий")
print("  - Безпека: ~128 біт (класична) + 128 біт (постквантова)")
print("  - Перевага: Безпека навіть якщо один алгоритм зламано")

print("\n" + "="*70)

ХАРАКТЕРИСТИКИ БЕЗПЕКИ

ECDSA P-256 (NIST FIPS 186-5):
   - Крива: NIST P-256 (secp256r1)
   - Поле: GF(p) де p ≈ 2^256
   - Хеш: SHA-256
   - Класична безпека: ~128 біт
   - Квантова вразливість: Повністю зламується (Шор)
   - Стандарт: Міжнародний NIST

ML-DSA-44 (NIST FIPS 204):
   - Базис: Module Lattice
   - Рівень: NIST Level 2
   - Безпека: ~128 біт класичної
   - Стійкість: Квантові атаки (128 біт)
   - Стандарт: FIPS 204 (2024)

Гібридна схема:
  - Верифікація: ОБИДВА підписи мають бути валідні
  - Захист: Класичний + Постквантовий
  - Безпека: ~128 біт (класична) + 128 біт (постквантова)
  - Перевага: Безпека навіть якщо один алгоритм зламано



## 10. Очищення

In [10]:
import os

# Видаляємо тестовий файл
try:
    if os.path.exists('signature_demo.json'):
        os.remove('signature_demo.json')
        print(" Тестові файли видалено")
except Exception as e:
    print(f"  {e}")

print("\n" + "="*70)
print(" ДЕМОНСТРАЦІЯ ЗАВЕРШЕНА УСПІШНО!")
print("="*70)

 Тестові файли видалено

 ДЕМОНСТРАЦІЯ ЗАВЕРШЕНА УСПІШНО!
